In [44]:
import pandas as pd
import numpy as np

Inner merge(commits,issue,on="issue_id",how="inner") korar por new_data.csv file asche

In [20]:
df=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\new_data.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_17848\790857003.py:1: DtypeWarning: Columns (24,27,29,30,31,33,35,39,40,41,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\new_data.csv')


In [22]:
df.columns

Index(['Unnamed: 0', 'source_commit', 'repo', 'hash', 'parents', 'author',
       'author_time', 'author_tz', 'committer', 'commit_time', 'commit_tz',
       'message', 'added', 'removed', 'from_svn', 'accurate_tz', 'issue_id',
       'message_sentiment', 'local_commit_time', 'local_author_time',
       'part_of_day_commit', 'part_of_day_author', 'source_issue', 'product',
       'issue_key', 'created', 'updated', 'last_resolved', 'summary',
       'description', 'version', 'milestone', 'status', 'severity', 'priority',
       'issuetype', 'resolution', 'component', 'votes', 'product_name',
       'reporter_key', 'reporter_tz', 'creator_key', 'creator_tz',
       'assignee_key', 'assignee_tz', 'summary_sentiment'],
      dtype='object')

In [23]:
df = df[(df['source_commit'] != 'mozilla') & (df['source_issue'] != 'mozilla')]


In [24]:
len(df['product'].unique())

428

In [31]:
df['source_issue'].unique()

array(['apache'], dtype=object)

In [41]:
df3=df[['repo','committer','commit_time','part_of_day_commit','issue_id','issuetype','priority','message','message_sentiment']]

In [42]:
df3.columns

Index(['repo', 'committer', 'commit_time', 'part_of_day_commit', 'issue_id',
       'issuetype', 'priority', 'message', 'message_sentiment'],
      dtype='object')

In [43]:
df3.head()

,repo,committer,commit_time,part_of_day_commit,issue_id,issuetype,priority,message,message_sentiment
0,accumulo-bsp,58d05bc1bf093c031c0caead30d37b44c48bdaf1,2013-01-15 23:58:38+00:00,Night,12550786.0,New Feature,Major,ACCUMULO-532 Update contrib to reflect changes...,neutral
1,accumulo-bsp,58d05bc1bf093c031c0caead30d37b44c48bdaf1,2013-01-22 18:08:30+00:00,Evening,12608497.0,Bug,Minor,ACCUMULO-769 Fix bsp contrib build due to mapr...,neutral
2,accumulo-bsp,83273207ef443072fc57aa858fff07fd5697a4a8,2012-05-17 17:24:30+00:00,Afternoon,12550786.0,New Feature,Major,ACCUMULO-532 added IOException to constructor ...,neutral
3,accumulo-bsp,83273207ef443072fc57aa858fff07fd5697a4a8,2012-05-16 14:51:58+00:00,Afternoon,12556041.0,Bug,Major,ACCUMULO-593 restructured contrib git-svn-id: ...,neutral
4,accumulo-bsp,83273207ef443072fc57aa858fff07fd5697a4a8,2012-05-16 17:42:07+00:00,Afternoon,12556041.0,Bug,Major,ACCUMULO-593 added license header to poms git-...,neutral


In [35]:
total_repos = df3['repo'].nunique()

# Calculate total number of committers
total_committers = df3['committer'].nunique()

# Calculate total number of commits
total_commits = df3['commit_time'].count()

# Calculate total number of issues
total_issues = df3['issue_id'].nunique()

# Display the results
print(f"Total Repositories: {total_repos}")
print(f"Total Committers: {total_committers}")
print(f"Total Commits: {total_commits}")
print(f"Total Issues: {total_issues}")

Total Repositories: 1145
Total Committers: 7680
Total Commits: 741272
Total Issues: 336831


In [36]:
commits_per_committer = df3.groupby('repo')['commit_time'].count().reset_index(name='commit_count')

# Sort the committers by the number of commits in descending order
commits_per_committer_sorted = commits_per_committer.sort_values(by='commit_count', ascending=False)

# Select the top 10 committers
top_10_committers = commits_per_committer_sorted.head(10)

# Display the top 10 committers
print("Top 10 Committers with Highest Commits:")
print(top_10_committers)

Top 10 Committers with Highest Commits:
                   repo  commit_count
355              hadoop         56396
364               hbase         41348
532         lucene-solr         39820
35               ambari         34297
116               camel         26761
1088              spark         26003
118   camel-spring-boot         19884
372                hive         18093
330               flink         14326
716             phoenix         10888


In [48]:
df3.loc[:, 'commit_time'] = pd.to_datetime(df3['commit_time'])

# Calculate the total number of days for each repository
repo_durations = df3.groupby('repo')['commit_time'].agg(lambda x: (x.max() - x.min()).days + 1).reset_index(name='total_days')

# Calculate commits per day for each developer
df3.loc[:, 'commit_date'] = df3['commit_time'].dt.date
commits_per_day = df3.groupby(['repo', 'committer', 'commit_date']).size().reset_index(name='commits')

# Function to calculate regularity index
def calculate_regularity_index(committer_data, total_days):
    di = committer_data['commit_date'].nunique()
    C = committer_data['commits'].sum()
    avg_commits_per_day = C / di
    commit_sd = np.sqrt((1 / di) * np.sum((committer_data['commits'] - avg_commits_per_day) ** 2))
    R = (di / total_days) * (1 / (commit_sd + 1))
    return pd.Series([R, di, C], index=['regularity_index', 'active_days', 'total_commits'])

# Apply the function to each group of committer and repo
regularity_indices = commits_per_day.groupby(['repo', 'committer'], group_keys=False).apply(
    lambda x: calculate_regularity_index(x, repo_durations.loc[repo_durations['repo'] == x['repo'].iloc[0], 'total_days'].values[0])
).reset_index()

# Classify committers into regular, average, and irregular
Q1 = regularity_indices['regularity_index'].quantile(0.25)
Q3 = regularity_indices['regularity_index'].quantile(0.75)
IQR = Q3 - Q1

regularity_indices['category'] = 'medium'
regularity_indices.loc[regularity_indices['regularity_index'] <= Q1, 'category'] = 'low'
regularity_indices.loc[regularity_indices['regularity_index'] >= Q3, 'category'] = 'high'

# Merge all classifications back into df4
df4 = df3.merge(regularity_indices[['repo', 'committer', 'category']], on=['repo', 'committer'], how='left')

# Save the updated dataframe to a new CSV file
df4.to_csv('commits_with_regularity_index.csv', index=False)

C:\Users\User\AppData\Local\Temp\ipykernel_17848\1665033498.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regularity_indices = commits_per_day.groupby(['repo', 'committer'], group_keys=False).apply(


In [49]:
df4.head()

,repo,committer,commit_time,part_of_day_commit,issue_id,issuetype,priority,message,message_sentiment,commit_date,category
0,accumulo-bsp,58d05bc1bf093c031c0caead30d37b44c48bdaf1,2013-01-15 23:58:38+00:00,Night,12550786.0,New Feature,Major,ACCUMULO-532 Update contrib to reflect changes...,neutral,2013-01-15,high
1,accumulo-bsp,58d05bc1bf093c031c0caead30d37b44c48bdaf1,2013-01-22 18:08:30+00:00,Evening,12608497.0,Bug,Minor,ACCUMULO-769 Fix bsp contrib build due to mapr...,neutral,2013-01-22,high
2,accumulo-bsp,83273207ef443072fc57aa858fff07fd5697a4a8,2012-05-17 17:24:30+00:00,Afternoon,12550786.0,New Feature,Major,ACCUMULO-532 added IOException to constructor ...,neutral,2012-05-17,high
3,accumulo-bsp,83273207ef443072fc57aa858fff07fd5697a4a8,2012-05-16 14:51:58+00:00,Afternoon,12556041.0,Bug,Major,ACCUMULO-593 restructured contrib git-svn-id: ...,neutral,2012-05-16,high
4,accumulo-bsp,83273207ef443072fc57aa858fff07fd5697a4a8,2012-05-16 17:42:07+00:00,Afternoon,12556041.0,Bug,Major,ACCUMULO-593 added license header to poms git-...,neutral,2012-05-16,high
